---
## Table of Contents

- [__Part 0: Import statements__](#0.)
- [__Part 1: Data gathering and processing__](#1.)
    - [Find and collect the nodes ](#1.1.)
    - [Find and collect episode metadata](#1.2.)
    - [Find and collect story lines](#1.3.)
    - [Transscript processing](#1.4.) 
    - [Scene data processing](#1.5.)
    - [Who talks about who? Collecting the data](#1.6.)
- [__Part 2: Create dataframe with attributes__](#2.)
- [__Part 3: Word Clouds data preprocessing__](#3.)

<a id='0.'></a>
# Import statments

This section contains all the import statements for the notebook. A requriments.txt file is also provided with specific package versions.

In [1]:
# Import nessary packages: 
import numpy as np 
import pandas as pd 
import networkx as nx
import matplotlib.pyplot as plt
import nltk
from os import listdir
from os.path import isfile, join
from wordcloud import WordCloud
from operator import itemgetter
import random
import os
from fa2 import ForceAtlas2
from matplotlib.pyplot import figure
import powerlaw
import networkx.algorithms.community as nx_comm
import json
import collections
from nltk.tokenize import sent_tokenize,word_tokenize, RegexpTokenizer 
from labMTsimple.storyLab import emotionFileReader, emotion, stopper, emotionV
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import words
import re
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import itertools
import urllib
import imdb
import json
from urllib.parse import quote
import ast
from SPARQLWrapper import SPARQLWrapper, JSON
from fuzzywuzzy import fuzz
import operator
import csv
from matplotlib.pyplot import figure
from gensim.models import LdaModel
from pprint import pprint
from scipy import stats
import pprint
import matplotlib.pyplot as plt
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
import gender_guesser.detector as gender


/root/venv/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


<a id='1.'></a>
# Data gathering and preprocessing

<a id='1.1.'></a>
## Find the nodes 

In [2]:
# URL for Friends transscripts
transscripts_url = "https://fangj.github.io/friends/"

# Working directory path
wd = os.getcwd()

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
# Function for creating the nodes
def CreateNodesCSV():

    # First check if file already exists
    path = "nodes.csv"
    if os.path.exists(path) == True: return

    print("Creating nodes files")

    # Friends characters are retrived from the Fandom Wiki
    characterList_url = "https://friends.fandom.com/api.php?action=query&format=json&list=categorymembers&cmtitle=Category:Friends_Characters&cmlimit=max&format=json"

    with urllib.request.urlopen(characterList_url) as response:
        html = response.read()

    data = json.loads(html)
    path = data["query"]["categorymembers"]

    list_of_characters = []

    for p in path:
        name = p.get("title")
        
        # Filtering sub-categories and list away 
        if (not name.startswith("Category") and not name.startswith("List")):
            
            # Data cleaning
            # We are just using the main characters first name
            if name == "Monica Geller": name = "Monica"
            elif name == "Joey Tribbiani": name = "Joey"
            elif name == "Phoebe Buffay": name = "Phoebe"
            elif name == "Rachel Greene": name = "Rachel"
            elif name == "Ross Geller": name = "Ross"
            elif name == "Chandler Bing": name = "Chandler"

            # Add name to the list of named characters
            list_of_characters.append(name)

    # Write nodes to CSV
    # The CSV file is used both for analysis and the project website
    df_characters = pd.DataFrame(list_of_characters)
    df_characters.columns = ['Name']

    # Write the list of chracters to a CSV file, without the index column
    df_characters.to_csv("nodes.csv", index=False)
    

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
CreateNodesCSV()

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Find and collect the edges

In [5]:
# Function for downloading character pages from Fandom wiki
def download_pages(title):

    #path = os.getcwd()
    title_new = "titles=" + quote(title)
    baseurl  = "https://friends.fandom.com/api.php?"
    characterList_url = "https://friends.fandom.com/api.php?action=query&format=json&list=categorymembers&cmtitle=Category:Friends_Characters&cmlimit=max&format=json"
    action = "action=query"
    content = "rvslots=*&prop=revisions&rvprop=content"
    dataformat = "format=json"
    query = "%s%s&%s&%s&%s" % (baseurl,action,title_new,content,dataformat)

    response = urllib.request.urlopen(query)
    source = response.read()
    json_character = json.loads(source)
    
    # Data cleaning
    if title == 'David "Tank" Abbott':
        p = "Pages/David_Tank_Abbott" + '.json'
        with open(p, 'w') as json_file:
            json.dump(json_character, json_file)
    elif title == 'Mike "Gandalf" Ganderson':
        p = "Pages/Mike_Gandalf_Ganderson" + '.json'
        with open(p, 'w') as json_file:
            json.dump(json_character, json_file)
    elif title == 'Restaurant Manger/Producer':
        p = "Pages/Restaurant_Manger_Producer" + '.json'
        with open(p, 'w') as json_file:
            json.dump(json_character, json_file)
    elif title == 'Roy "Goodbody"':
        p = "Pages/Roy_Goodbody" + '.json'
        with open(p, 'w') as json_file:
            json.dump(json_character, json_file)
    else:
        p = "Pages/" + title.replace(' ', '_') + '.json' 
        with open(p, 'w') as json_file:
            json.dump(json_character, json_file)
    

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
# Download the character pages
nodes = pd.read_csv('nodes.csv')
list_of_characters = list(nodes['Name'])
for title in list_of_characters:
    download_pages(title)

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
# Read the character wiki pages
raw_content = {} 
# the path must consist of the local project path to which the Pages directory must be added 
path = os.getcwd() + '//Pages'
os.chdir(path)

for name in list_of_characters:
    #get character data one by one
    
    if name == 'David "Tank" Abbott':
        name_new = "David Tank Abbott"
        f = open(name_new.replace(' ','_')+'.json')
        character_data = f.read()
        raw_content[name] = character_data
        
    elif name == 'Mike "Gandalf" Ganderson':
        name_new = "Mike Gandalf Ganderson"
        f = open(name_new.replace(' ','_')+'.json')
        character_data = f.read()
        raw_content[name] = character_data
        
    elif name == 'Restaurant Manger/Producer':
        name_new = "Restaurant Manger Producer"
        f = open(name_new.replace(' ','_')+'.json')
        character_data = f.read()
        raw_content[name] = character_data
        
    elif name == 'Roy "Goodbody"':
        name_new = 'Roy Goodbody'
        f = open(name_new.replace(' ','_')+'.json')
        character_data = f.read()
        raw_content[name] = character_data
        
    else:
    
        f = open(name.replace(' ','_')+'.json')
        character_data = f.read()
        raw_content[name] = character_data

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
# Get content of each character wiki page
characters_content = {}
for key in list(raw_content.keys()):
    from_string = ast.literal_eval(re.search('({.+})', raw_content[key]).group(0))
    
    k = list(from_string['query']['pages'].keys())[0]
    page_content = from_string['query']['pages'][k]['revisions'][0]['slots']['main']['*']
    
    characters_content[key] = page_content 

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [9]:
#for each character in the list of characters, get content 
#find in content the list of the names 

ch_and_connections = {}
for ch in list_of_characters:
    content = characters_content[ch]
    name_var = ch.split()
    connections = [con for con in list_of_characters if con in content and con!=ch]
    ch_and_connections[ch] = connections

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [10]:
list_of_lists = []
for key in list(ch_and_connections.keys()):
    
    con_lst = ch_and_connections[key]
    con_each_ch = []
    for con in con_lst:
        con_each_ch.append((key,con))
    list_of_lists.extend(con_each_ch)

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
friends_links = set()
temp = [friends_links.add((a, b)) for (a, b) in list_of_lists if (a, b) and (b, a) not in friends_links]

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
# Create a DataFrame of the list of links
df_links = pd.DataFrame(list(friends_links), columns =['CH_1', 'CH_2'])

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [13]:
print("Number of edges used for network #1:", len(df_links))

Number of edges used for network #1: 2183
/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [14]:
# Revert the working directory back
os.chdir("..")

# Create a CSV file that contains the edges for the network
df_links.to_csv('friends_links.csv', index= False)

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Find and collect episode metadata

In [15]:
# Director and Writers for each episodes cannot be retrieved using IMDBPy
# https://github.com/alberanid/imdbpy/issues/328

def CreateEpisodesCSV():
    transscripts_url = "https://fangj.github.io/friends/"
    path = "episodes.csv"

    ia = imdb.IMDb()

    # 0108778 is the IMDB id for the Friends series
    series = ia.get_movie('0108778')
    # add informations about the episodes
    ia.update(series, 'episodes')   

    # First check if the file already exists
    if os.path.exists(path) == True: return

    print("Creating episodes csv file")

    with urllib.request.urlopen(transscripts_url) as response:
        text = response.read()

    episodes = []

    matches = re.finditer(r'<a href="season\/(?P<filename>[0123456789-]*.html)">(?P<title>.*?)<\/a>', str(text))

    for match in matches:
        url = match.group('filename')
        title = match.group('title')
        #print(name, " ", title)
        print(title)
        cc = title.split(' ')[0]
        if (len(cc) == 4):
            season = int(title[:2])
            episode = int(title[2:4])
        else:
            season = int(title[:1])
            episode = int(title[1:3]) 
        #episode = ia.search_movie(title)
        print(season, " - ", episode )
        imdb_episode = series['episodes'][season][episode]
        rating = imdb_episode.data["rating"]

        dne = [item for item in directorList if (fuzz.ratio(title,str(item[0])) > 50)]

        directorname = ""
        if (len(dne) > 0):
            directorname = dne[0][1]
            print(dne)

        episodes.append((url,title,rating, directorname))
    
    df_episodes = pd.DataFrame(episodes)
    df_episodes.columns = ['Filename', 'Title', 'Rating', 'Director']
    df_episodes.to_csv(path, index= False )
    

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [16]:
# Call the function to create nodes for the network

CreateEpisodesCSV()

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [17]:
# Get director information from Wikidata

sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
sparql.setQuery("""
SELECT ?episodename ?director ?directorname
WHERE 
{
  ?item wdt:P179 wd:Q79784
  OPTIONAL{
    ?item wdt:P57 ?director
    OPTIONAL{?director rdfs:label ?directorname}
    OPTIONAL{?item rdfs:label ?episodename} 
  }  
  FILTER (LANG(?directorname) = "en")
  FILTER (LANG(?episodename) = "en")
}
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

directorList = []

for epi in results["results"]["bindings"]:
  episodename = epi["episodename"]["value"]
  directorname = epi["directorname"]["value"] 
  directorList.append((episodename,directorname))

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Find and collect story lines 

In [18]:
# Function that extracts the sentences from html content
# The transscripts syntax is not consistent. We are therefor using two differnt 
# regex patterns to retrive the needed information

def getsentences(text):

    sentences = []

    # Type 1 markup

    matches = re.finditer(r'<b>(?P<name>[\w]+):<\/b>[ ]*(?P<content>[\w\s\\n\'.,?$!]+)', text, flags=re.IGNORECASE | re.MULTILINE)

    for match in matches:
        name = match.group('name')
        #print(name)
        content = match.group('content') 
        content = content.replace('\\n', ' ') 
        content = content.replace('\\', '') 

        sentences.append((name,content))

    # Type 2 mark up
    matches = re.finditer(r'<strong>(?P<name>[\w]+)<\/strong>:[ ]*(?P<content>[\w\s\\n\'.,?$!]+)', text, flags=re.IGNORECASE | re.MULTILINE)

    for match in matches:
        name = match.group('name')
        content = match.group('content') 
        content = content.replace('\\', '')

        sentences.append((name,content))

    # return the list of sentences
    return sentences


/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [19]:
# Read characters from CSV
characters_path = "nodes.csv"
df_characters = pd.read_csv(characters_path)
print("Number of characters:", len(df_characters))

Number of characters: 423
/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [20]:
# Read episodes from CSV
episodes_path = "episodes.csv"
df_episodes = pd.read_csv(episodes_path) 

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Transscript processing

In [21]:
# Get all sentences for all episodes

total =[]

for index, e in df_episodes.iterrows():
#for e in episode_list:
    filename = e["Filename"]
    url = transscripts_url + "season/" + filename
    #print(filename)

    #get entire episode content 
    with urllib.request.urlopen(url) as response:
        html3 = response.read()

    sl = getsentences(str(html3))

    for s in sl:
        name = s[0]
        content = s[1]
        episode = e

        total.append((name,content, episode))

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [22]:
print("Total number of transscript story lines:", len(total))

Total number of transscript story lines: 46657
/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


<a id='1.2.'></a>
## Scene data processing 

In [23]:
scenes_per_episode = {}
for index, e in df_episodes.iterrows():
    url = transscripts_url + "season/" + e[0]
    #print(e[1]," ",url)
    
    #get entire episode content 
    with urllib.request.urlopen(url) as response:
        html3 = response.read()
    
    scences = re.findall(r'\[Scene(.*?)\]', str(html3))
    
    episode_scenes = []
    for scene in scences:
        
        scene_clean = scene.replace('\\n',' ')
        scene_clean_v2 = scene_clean.replace( '\\','')
        episode_scenes.append(scene_clean_v2)
    scenes_per_episode[e[0]] = episode_scenes

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [24]:
list_of_characters_v2 = df_characters['Name'].tolist()

characters_all_episodes = {}

for e in list(scenes_per_episode.keys()):
    
    episode = scenes_per_episode[e]
    
    characters_in_episode = []
    #characters_in_episode_v2 = []
    for scene in episode:

        characters_in_scene = []
        #loop through list of characters
        for character in list_of_characters_v2:
            #find characters in scene 
            if character in scene:

                characters_in_scene.append(character)
        characters_in_episode.append(list(set(characters_in_scene)))
        
        characters_in_episode_v2 = [l for l in characters_in_episode if len(l) > 1 ]
            
    characters_all_episodes[e] = characters_in_episode_v2

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [25]:
#generate unique list of tuples based on the characters extracted from each scene
links = {}
for e in list(characters_all_episodes.keys()):
    
    episode = characters_all_episodes[e]
    
    res_tuples = set()
    links_in_episode = []
    for l in episode:
        #tuples for each list(scene) in the episode
        all_tuples = [(a,b) for a in l for b in l if a!=b]
        temp = [res_tuples.add((a, b)) for (a, b) in all_tuples if (a, b) and (b, a) not in res_tuples]
        #print(all_tuples)
        links_in_episode.append(list(res_tuples))
    links[e] = links_in_episode


/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [26]:
all_links = list(links.values())
all_links_flat = [item for sublist in all_links for item in sublist]
#flat_list = [item for sublist in t for item in sublist]
all_links_flat_v2 = [item for sublist in all_links_flat for item in sublist]
all_links_final = set()
temp = [all_links_final.add((a, b)) for (a, b) in all_links_flat_v2 if (a, b) and (b, a) not in all_links_final]

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [27]:
def CreateEdgesCSV():

    path_edges = "edges.csv"

    # First check if the file already exists
    if os.path.exists(path_edges) == True: return

    print("Creating edges csv file")

    df_edges = pd.DataFrame(all_links_final)
    
    df_edges.columns = ['From','To']

    # Write the list of chracters to a CSV file, without the index column
    df_edges.to_csv(path_edges, index=False)


/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [28]:
CreateEdgesCSV()

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


<a id='1.3.'></a>
## Who talks about who? Collecting the data  

In [29]:
# Find subjects in text
# Define a DataFrame to hold edges
column_names = ["From", "To"]

df = pd.DataFrame(columns=column_names)

for a in total:
    character = a[0]
    content = a[1]

    for c in list_of_characters_v2:
        # look for this character in content
        if c in content:
            # we have an edge
            if (character != c):
                df.loc[len(df.index)] = [character, c]

df = df.drop_duplicates()

df.to_csv("edges_talkabout.csv", index=False)

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [30]:
# Almost the same as above, but now including storylines
# Define a DataFrame to hold edges
column_names = ["Speaker", "About", "Sentence"]

df_talkabout2 = pd.DataFrame(columns=column_names)

for a in total:
    character = a[0]
    content = a[1]

    for c in list_of_characters_v2:
        # look for this character in content
        if c in content:
            # we have an edge

            # Check that it is not a reference to it self
            if (character != c):
                df_talkabout2.loc[len(df_talkabout2.index)] = [character, c, content]

df_talkabout2.to_csv("edges_talkabout2.csv", index=False)


/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [31]:
count = 0
for a in total:
    character = a[0]
    content = a[1]

    if (character == "Ross" or character == "Joey"):
        if "dude" in content.lower():
            count += 1

print(count)


62
/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


<a id='2.'></a>
## Create dataframe with attributes

In [32]:
characters_sentiment = pd.read_csv('characters_sentiment.csv').drop_duplicates()
characters_sentiment = characters_sentiment[['Character','Sentiment']]

characters = characters_sentiment['Character'].to_list()
sentiment = characters_sentiment['Sentiment'].to_list()

sentiment_vals = {characters[i]: sentiment[i] for i in range(len(characters))}

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [33]:
ch_and_lines = pd.read_csv('lines_and_words_agg.csv')
ch_and_lines = ch_and_lines [['Character','Lines','no_sentences','no_words']]

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [34]:
nodes = pd.read_csv('nodes.csv')
main_ch = ['Ross','Monica','Chandler','Phoebe','Joey','Rachel']
all_ch = list(set(sentiment_vals.keys())-set(main_ch))
ch_not_matched = [] 

for ch in all_ch:
    if not ch in list(nodes.Name):
        ch_not_matched.append(ch)
nodes_with_attributes = nodes.copy()

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [35]:
#add sentiment to dataframe 
for i in range(0,len(nodes_with_attributes)):    
    #loop through sent dictionary 
    full_name = nodes_with_attributes.iloc[i].values[0]
    
    for ch in all_ch:
        if ch in full_name:
            nodes_with_attributes.at[i,'Sentiment'] = sentiment_vals[ch]
#now add sentiment for main ch 

for name in main_ch:
    idx =  nodes_with_attributes[nodes_with_attributes['Name'] == name].index[0]
    nodes_with_attributes.at[idx,'Sentiment'] = sentiment_vals[name]

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [36]:
#add number of sentences for all other characters other than the main ones 
for i in range(0,len(nodes_with_attributes)):
    full_name = nodes_with_attributes.iloc[i].values[0]
    
    if not ch_and_lines[ch_and_lines['Character'] == full_name ]['no_sentences'].values:
        nodes_with_attributes.at[i,'No_of_Lines'] = 0 
    else:
        nodes_with_attributes.at[i,'No_of_Lines'] = ch_and_lines[ch_and_lines['Character'] == full_name ]['no_sentences'].values[0]

for name in main_ch:
    idx =  nodes_with_attributes[nodes_with_attributes['Name'] == name].index[0]
    nodes_with_attributes.at[idx,'No_of_Lines'] = ch_and_lines[ch_and_lines['Character'] == name ]['no_sentences'].values[0]

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  """
/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  """
/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_lau

In [37]:
## add gender 
d = gender.Detector()
for i in range(0,len(nodes_with_attributes)):
    
    full_name = nodes_with_attributes.iloc[i].values[0]
    names_matches = full_name.split()
    for name in names_matches:
        if d.get_gender(name) != 'unknown':
            nodes_with_attributes.at[i,'Gender'] = d.get_gender(name)
    # remove the word mostly from the gender column 
for i in range(0,len(nodes_with_attributes)-1):
    g = nodes_with_attributes.iloc[i]['Gender']
    if type(g) == str:
        nodes_with_attributes.at[i,'Gender'] = g.replace('mostly_','')

        
nodes_with_attributes['Gender'] = nodes_with_attributes['Gender'].fillna('unknown')
nodes_with_attributes['Gender'] = np.where(nodes_with_attributes['Gender'] == 'andy', 'unknown', nodes_with_attributes['Gender'])
nodes_with_attributes['Gender'] = np.where(nodes_with_attributes['Name'].str.contains('Mr.'),'male',nodes_with_attributes['Gender'])
nodes_with_attributes['Gender'] = np.where(nodes_with_attributes['Name'].str.contains('Mrs.'),'female',nodes_with_attributes['Gender'])

nodes_with_attributes['Gender'] = np.where(nodes_with_attributes['Name'].str.contains('Stryker|Todd|Allessandro|Charles|grandfather|boyfriend|husband|Male|stepfather|father'),'male',nodes_with_attributes['Gender'])
nodes_with_attributes['Gender'] = np.where(nodes_with_attributes['Name'].str.contains('Cailin|Margha|Erica|Frannie|Ms.|daughter|wife|Grandma|girlfriend|Mary-Therese|Mary-Angela|Nora Tyler Bing|mother|Sisters'),'female',nodes_with_attributes['Gender'])


nodes_with_attributes.to_csv('nodes_with_attributes.csv')

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [38]:
len(nodes_with_attributes[nodes_with_attributes.Gender == 'unknown'])

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


36

<a id='3.'></a>
## Wordclouds data

In [39]:
episode_sentences_list = []

for index,e in df_episodes.iterrows():
    url = transscripts_url + "season/" + e[0]
    # print(e[1]," ",url)

    with urllib.request.urlopen(url) as response:
        html3 = response.read()
        html3_processed = str(html3).replace('\\n',' ')
        
    sentences_list = getsentences(str(html3_processed).replace('\\',''))
    #sentences_list = getsentences(str(html3))

    episode_sentences_list.append((e,sentences_list))

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [40]:
# Make files with sentence data
# One file per character
champions = ["Chandler","Joey", "Monica", "Phoebe", "Ross", "Rachel"]

# For each episode in the list
if len(os.listdir('Content')) == 0:
    for sf in episode_sentences_list:
        episodeInfo = sf[0]
        epsiode_sentences = sf[1]

        for s in epsiode_sentences:
            # s = sentence in one episode
            name = s[0]  # name of the character saying the sentence
            text = s[1]  # text of the sentence

            
            if name in champions:
                    pattern = "[^\w\s]"
                    #text = str(sl[1].split(' '))
                    text = re.sub(pattern, ' ', text)
                    text = re.sub("  ", ' ', text)
                        
                    # save file
                    p = "Content/" + name + ".txt"
                    #print(p)

                    # add space to seperate sentences
                    text = text + " "

                    with open(p, 'a', encoding='UTF8', newline='') as f:
                        f.write(text)

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<>:18: DeprecationWarning: invalid escape sequence \w
<>:18: DeprecationWarning: invalid escape sequence \w
<>:18: DeprecationWarning: invalid escape sequence \w
<ipython-input-40-820bb2f6586a>:18: DeprecationWarning: invalid escape sequence \w
  pattern = "[^\w\s]"


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=aee39066-d006-4842-a0e1-46c68e5ff786' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>